In [1]:
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,random_split
from torchvision import datasets, transforms

import pandas as pd
import numpy as np

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [3]:
data = pd.read_csv("dummyData-0.csv",header = None)
x=data.iloc[1:,1:5].astype('float32')
y=data.iloc[1:,-1].astype('float32')
# x.iloc[:, 0] = x.iloc[:, 0] / 23
x.iloc[:, 2] = x.iloc[:, 2] / 100
x.iloc[:, 3] = x.iloc[:, 3] / 100
y = y/max(y.values)

In [4]:
x

,1,2,3,4
1,0.0,1.0,0.14,0.65
2,1.0,1.0,0.74,0.15
3,0.0,0.0,0.48,0.07
4,0.0,1.0,0.45,0.40
5,1.0,0.0,0.99,0.30
...,...,...,...,...
99996,0.0,1.0,0.91,0.01
99997,0.0,1.0,0.74,0.84
99998,1.0,1.0,0.77,0.84
99999,1.0,1.0,0.98,0.69


In [5]:
class WaterLevelDataset(Dataset):
    def __init__(self, x, y):
        x= torch.tensor(x,dtype=torch.float32)
        y = torch.tensor(y,dtype=torch.float32)
        self.X = x
        self.Y = y

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.Y[idx]]

In [6]:
dataset = WaterLevelDataset(x.values,y.values)

In [7]:
len(dataset.X)

100000

In [8]:
dataset.X

tensor([[0.0000, 1.0000, 0.1400, 0.6500],
        [1.0000, 1.0000, 0.7400, 0.1500],
        [0.0000, 0.0000, 0.4800, 0.0700],
        ...,
        [1.0000, 1.0000, 0.7700, 0.8400],
        [1.0000, 1.0000, 0.9800, 0.6900],
        [1.0000, 0.0000, 0.5100, 0.4600]])

In [9]:
dataset.Y

tensor([0.0000, 0.0333, 0.2333,  ..., 0.0000, 0.8333, 0.1333])

In [10]:
train, test = random_split(dataset, [int(len(dataset.X)*0.8),int(len(dataset.X)*0.2)])

In [11]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

## Model 1
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [12]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=self.layer2(X)
        X = self.activation(X)
        return X

# Instantiate the model
model = SimpleNN().to(device)



In [13]:
loss_fn = nn.MSELoss().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001)

In [14]:
for epoch in range(100):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 9.997019
Epoch [2/100], Loss: 5.523160
Epoch [3/100], Loss: 3.751296
Epoch [4/100], Loss: 6.221991
Epoch [5/100], Loss: 16.333282
Epoch [6/100], Loss: 4.655921
Epoch [7/100], Loss: 3.357554
Epoch [8/100], Loss: 4.403000
Epoch [9/100], Loss: 9.782785
Epoch [10/100], Loss: 6.935818
Epoch [11/100], Loss: 5.780757
Epoch [12/100], Loss: 2.977354
Epoch [13/100], Loss: 10.147522
Epoch [14/100], Loss: 4.654537
Epoch [15/100], Loss: 6.189547
Epoch [16/100], Loss: 10.745932
Epoch [17/100], Loss: 4.331743
Epoch [18/100], Loss: 6.903541
Epoch [19/100], Loss: 5.352234
Epoch [20/100], Loss: 6.079121
Epoch [21/100], Loss: 3.190345
Epoch [22/100], Loss: 1.181488
Epoch [23/100], Loss: 7.335351
Epoch [24/100], Loss: 6.662310
Epoch [25/100], Loss: 3.992809
Epoch [26/100], Loss: 3.182200
Epoch [27/100], Loss: 5.992000
Epoch [28/100], Loss: 2.353135
Epoch [29/100], Loss: 5.393059
Epoch [30/100], Loss: 4.488914
Epoch [31/100], Loss: 5.598217
Epoch [32/100], Loss: 6.139958
Epoch [33/100]

In [15]:
model.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = model(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx].to(device))
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")


Predicted: 3.37, Actual: 8.00
Predicted: 1.23, Actual: 8.00
Predicted: 1.93, Actual: 0.00
Predicted: 17.60, Actual: 26.00
Predicted: 14.01, Actual: 4.00
Predicted: 16.07, Actual: 29.00
Predicted: 4.61, Actual: 1.00
Predicted: 7.00, Actual: 5.00
Predicted: 22.91, Actual: 25.00
Predicted: 2.60, Actual: 0.00
Predicted: 5.15, Actual: 0.00
Predicted: 2.05, Actual: 10.00
Predicted: 5.07, Actual: 10.00
Predicted: 2.71, Actual: 2.00
Predicted: 21.11, Actual: 23.00
Predicted: 0.26, Actual: 0.00
Average loss on the testing dataset: 3.8558
Predicted: 6.53, Actual: 0.00
Predicted: 0.88, Actual: 0.00
Predicted: 3.45, Actual: 2.00
Predicted: 1.82, Actual: 5.00
Predicted: 1.73, Actual: 6.00
Predicted: 0.68, Actual: 0.00
Predicted: 8.56, Actual: 0.00
Predicted: 6.14, Actual: 7.00
Predicted: 10.33, Actual: 1.00
Predicted: 0.31, Actual: 0.00
Predicted: 1.06, Actual: 0.00
Predicted: 8.63, Actual: 4.00
Predicted: 3.77, Actual: 0.00
Predicted: 0.77, Actual: 1.00
Predicted: 7.50, Actual: 22.00
Predicted: 4.

In [16]:
torch.save(model, "model/model1Data10000-10.h5")

## Model 2
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [16]:
class Model2NN(nn.Module):
    def __init__(self):
        super(Model2NN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=torch.relu(self.layer2(X))
        X=self.layer3(X)
        X = self.activation(X)
        return X

# Instantiate the model
model1 = Model2NN().to(device)



In [17]:
loss_fn = nn.MSELoss().to(device)

optimizer = optim.SGD(model1.parameters(), lr=0.001)

In [18]:
for epoch in range(100):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model1(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 11.481173
Epoch [2/100], Loss: 7.291058
Epoch [3/100], Loss: 7.676771
Epoch [4/100], Loss: 3.083934
Epoch [5/100], Loss: 11.083514
Epoch [6/100], Loss: 4.668288
Epoch [7/100], Loss: 6.296626
Epoch [8/100], Loss: 8.296543
Epoch [9/100], Loss: 6.117584
Epoch [10/100], Loss: 2.813889
Epoch [11/100], Loss: 5.025133
Epoch [12/100], Loss: 4.830099
Epoch [13/100], Loss: 3.892322
Epoch [14/100], Loss: 8.053411
Epoch [15/100], Loss: 8.580338
Epoch [16/100], Loss: 5.736934
Epoch [17/100], Loss: 5.681282
Epoch [18/100], Loss: 5.766375
Epoch [19/100], Loss: 2.055185
Epoch [20/100], Loss: 6.045212
Epoch [21/100], Loss: 3.476299
Epoch [22/100], Loss: 6.495012
Epoch [23/100], Loss: 3.656743
Epoch [24/100], Loss: 2.939963
Epoch [25/100], Loss: 9.025745
Epoch [26/100], Loss: 2.174319
Epoch [27/100], Loss: 5.439377
Epoch [28/100], Loss: 4.064400
Epoch [29/100], Loss: 4.510383
Epoch [30/100], Loss: 6.755805
Epoch [31/100], Loss: 4.915097
Epoch [32/100], Loss: 6.446998
Epoch [33/100],

In [19]:
model1.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = model1(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx].to(device))
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")

Predicted: 3.24, Actual: 8.00
Predicted: 1.23, Actual: 8.00
Predicted: 1.63, Actual: 0.00
Predicted: 17.16, Actual: 26.00
Predicted: 14.74, Actual: 4.00
Predicted: 16.64, Actual: 29.00
Predicted: 4.74, Actual: 1.00
Predicted: 7.11, Actual: 5.00
Predicted: 21.66, Actual: 25.00
Predicted: 1.98, Actual: 0.00
Predicted: 5.14, Actual: 0.00
Predicted: 1.85, Actual: 10.00
Predicted: 5.50, Actual: 10.00
Predicted: 2.51, Actual: 2.00
Predicted: 21.24, Actual: 23.00
Predicted: 0.64, Actual: 0.00
Average loss on the testing dataset: 3.9418
Predicted: 6.46, Actual: 0.00
Predicted: 0.97, Actual: 0.00
Predicted: 3.20, Actual: 2.00
Predicted: 1.65, Actual: 5.00
Predicted: 1.68, Actual: 6.00
Predicted: 0.84, Actual: 0.00
Predicted: 6.65, Actual: 0.00
Predicted: 6.41, Actual: 7.00
Predicted: 10.35, Actual: 1.00
Predicted: 0.58, Actual: 0.00
Predicted: 0.93, Actual: 0.00
Predicted: 9.03, Actual: 4.00
Predicted: 3.37, Actual: 0.00
Predicted: 0.93, Actual: 1.00
Predicted: 7.83, Actual: 22.00
Predicted: 3.

In [32]:
torch.save(model1, "model/model2Data10000-10.h5")

In [1]:
test1 = torch.tensor([0,0,50/100,90/100]).to(device)
print(test1)
outputs = model1(test1)
print(int(outputs.item()*30))

NameError: name 'torch' is not defined